In [ ]:
import math
import numpy as np
from scipy.fft import fft, fftfreq
import matplotlib.pyplot as plt
import seaborn as sns

import sys
if '..' not in sys.path:
    sys.path = ['..'] + sys.path
from multitone import *

This notebook implements the algorithm described in the following paper:

S. Boyd, "Multitone signals with low crest factor," in _IEEE Transactions on Circuits and Systems_, vol. 33, no. 10, pp. 1018-1022, October 1986, doi: 10.1109/TCS.1986.1085837.

#### Compute the time series and crest factors for the first 100 tones

In [ ]:
F0 = 5e-3      # [Hz]
T = 1 / F0     # [s]
dt = T / 1000  # [s]
fs = 1 / dt    # [Hz]
N_T = 2
t = np.r_[0 : N_T * T + dt/2 : dt]
tend = t[-1]   # [s]
print(f'Simulation duration: {tend} s.')
N_tones = np.arange(1, 101)
U = {N: multitone(t, N, method='N', omega0=2 * np.pi * F0) for N in N_tones}
CF = {N: compute_crest_factor(u, 1 / fs) for N, u in U.items()}

#### Plot of the crest factor as a function of the number of tones

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5,3))
ax.plot(N_tones, 20 * np.log10(list(CF.values())), 'ko-', markerfacecolor='w', markersize=3)
ax.set_xlabel('No. of tones')
ax.set_ylabel('Crest factor [dB]')
sns.despine()
fig.tight_layout()

Plot the time series, its distribution and Fourier spectra for a number of tones equal to 32.

In [ ]:
N = 32
u = U[N] - U[N].mean()
print('Crest factor for no. of tones = {}: {:g} ({:g} dB).'.format(N, CF[N], 20 * np.log10(CF[N])))
n, x = compute_amplitude_distribution(u, bins='fd')

uf = fft(u)
ups = 2 / u.size * np.abs(uf[:u.size // 2])
freq = fftfreq(u.size, 1 / fs)[:u.size//2]
ω = 2 * np.pi * freq

F = np.array([(i + 1) * F0 for i in range(N)])
γc, γs = np.empty(N), np.empty(N)
ttran = 0
for i, f in enumerate(F):
    k = i + 1
    N_periods = (tend - ttran) * f
    δ = newman_phase(k, N)
    γc[i] = f / N_periods * dt * np.cos(2 * np.pi * f * t + δ) @ u
    γs[i] = f / N_periods * dt * np.sin(2 * np.pi * f * t + δ) @ u

fig,ax = plt.subplots(3, 1, figsize=(5,5))
ax[0].plot(t, u, lw=0.75)
yl = ax[0].get_ylim()
ax[0].vlines(np.arange(N_T + 1) * T, yl[0], yl[1], color='tab:red', lw=0.75, ls=':')
ax[0].set_xlabel('Time [s]')
ax[0].set_ylabel('u')
# ax[0].set_xlim([0, 10])
ax[0].set_ylim([-2.5, 2.5])

ax[1].plot(x, n, 'k', lw=0.75)
ax[1].set_xlim([0, 2])
ax[1].set_ylim([0, 1])
ax[1].set_xlabel('Amplitude')
ax[1].set_ylabel(r'$F_u(a)$')

coeff = 1 if F0 > 0.1 else 1e3

ax[2].vlines(freq * coeff, 0 * ups, ups, lw=0.5)
ax[2].hlines(np.sqrt(2 / N), 0, (N + 2) * F0 * coeff, color='tab:red', ls='--', lw=1)
ax[2].plot(freq * coeff, ups, 'ko', markerfacecolor='w', markersize=3, lw=0.75)
ax[2].plot(F * coeff, np.abs(γc + 1j * γs), 'rs', markerfacecolor='w', markersize=3, lw=0.75)
ax[2].set_xlim([0, (N + 2) * F0 * coeff])
ax[2].set_xlabel('Frequency [{}Hz]'.format('m' if coeff == 1e3 else ''))
ax[2].set_ylabel('|FFT(u)|')

sns.despine()
fig.tight_layout()